In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time
import numpy as np
import librosa
import pickle
from numpy.lib import stride_tricks
import ipdb
import os
import itertools
import tensorflow as tf

C:\Users\91828\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\91828\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\91828\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\91828\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
FRAMES_PER_SAMPLE = 100  # number of frames forming a chunk of data
SAMPLING_RATE = 8000
FRAME_SIZE = 256
NEFF = 129  # effective FFT points
# amplification factor of the waveform sig
AMP_FAC = 10000
MIN_AMP = 10000
# TF bins smaller than THRESHOLD will be
# considered inactive
THRESHOLD = 40
# embedding dimention
EMBBEDDING_D = 40
# prams for pre-whitening
GLOBAL_MEAN = 44
GLOBAL_STD = 15.5
# feed forward dropout prob
P_DROPOUT_FF = 0.5
# recurrent dropout prob
P_DROPOUT_RC = 0.2

In [3]:
def stft(sig, frameSize, overlapFac=0.75, window=np.hanning):
    """ short time fourier transform of audio signal """
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    samples = np.array(sig, dtype='float64')
    # cols for windowing
    cols = int(np.ceil((len(samples) - frameSize) / float(hopSize)))
    frames = stride_tricks.as_strided(
        samples,
        shape=(cols, frameSize),
        strides=(samples.strides[0] * hopSize, samples.strides[0])).copy()
    frames *= win
    return np.fft.rfft(frames)


class DataGenerator(object):
    def __init__(self, data_dir, batch_size):
        '''preprocess the training data
        data_dir: dir containing the training data
                  format:root_dir + speaker_dir + wavfiles'''
        # get dirs for each speaker
        self.speakers_dir = [os.path.join(data_dir, i)
                             for i in os.listdir(data_dir)]
        self.n_speaker = len(self.speakers_dir)
        self.batch_size = batch_size
        self.speaker_file = {}
        self.epoch = 0

        # get the files in each speakers dir
        for i in range(self.n_speaker):
            wav_dir_i = [os.path.join(self.speakers_dir[i], file) for file in os.listdir(self.speakers_dir[i])]
            for j in wav_dir_i:
                if i not in self.speaker_file:
                    self.speaker_file[i] = []
                self.speaker_file[i].append(j)

    def resample(self):
        '''Resample all the files, not always necessary'''
        for speaker in self.speaker_file:
            for file in self.speaker_file[speaker]:
                data, sr = librosa.load(file, SAMPLING_RATE)
                librosa.output.write_wav(file, data, SAMPLING_RATE)

    def reinit(self, save_path):
        '''Init the training data using the wav files'''
        self.speaker_file_match = {}
        # generate match dict
        for i in range(self.n_speaker):
            for j in self.speaker_file[i]:
                k = np.random.randint(self.n_speaker)
                # requiring different speaker
                while(i == k):
                    k = np.random.randint(self.n_speaker)
                l = np.random.randint(len(self.speaker_file[k]))
                self.speaker_file_match[j] = self.speaker_file[k][l]

        self.samples = []
        self.ind = 0
        # for each file pair, generate their mixture and reference samples
        for i in self.speaker_file_match:
            j = self.speaker_file_match[i]
            speech_1, _ = librosa.core.load(i, sr=SAMPLING_RATE)
            # amp factor between -3 dB - 3 dB
            fac = np.random.rand(1)[0] * 6 - 3
            speech_1 = 10. ** (fac / 20) * speech_1
            speech_2, _ = librosa.core.load(j, sr=SAMPLING_RATE)
            fac = np.random.rand(1)[0] * 6 - 3
            speech_2 = 10. ** (fac / 20) * speech_2
            # mix
            length = min(len(speech_1), len(speech_2))
            speech_1 = speech_1[:length]
            speech_2 = speech_2[:length]
            speech_mix = speech_1 + speech_2
            # compute log spectrum for 1st speaker
            speech_1_spec = np.abs(stft(speech_1, FRAME_SIZE)[:, :NEFF])
            speech_1_spec = np.maximum(speech_1_spec, np.max(speech_1_spec) / MIN_AMP)
            speech_1_spec = 20. * np.log10(speech_1_spec * AMP_FAC)
            # same for the 2nd speaker
            speech_2_spec = np.abs(stft(speech_2, FRAME_SIZE)[:, :NEFF])
            speech_2_spec = np.maximum(speech_2_spec, np.max(speech_2_spec) / MIN_AMP)
            speech_2_spec = 20. * np.log10(speech_2_spec * AMP_FAC)
            # same for the mixture
            speech_mix_spec0 = stft(speech_mix, FRAME_SIZE)[:, :NEFF]
            speech_mix_spec = np.abs(speech_mix_spec0)
            speech_mix_spec = np.maximum(speech_mix_spec, np.max(speech_mix_spec) / MIN_AMP)
            speech_mix_spec = 20. * np.log10(speech_mix_spec * AMP_FAC)
            
            max_mag = np.max(speech_mix_spec)
            speech_VAD = (speech_mix_spec > (max_mag - THRESHOLD)).astype(int)
            speech_mix_spec = (speech_mix_spec - GLOBAL_MEAN) / GLOBAL_STD

            len_spec = speech_1_spec.shape[0]
            k = 0
            while(k + FRAMES_PER_SAMPLE < len_spec):
                sample_1 = speech_1_spec[k:k + FRAMES_PER_SAMPLE, :]
                sample_2 = speech_2_spec[k:k + FRAMES_PER_SAMPLE, :]
                sample_mix = speech_mix_spec[k:k + FRAMES_PER_SAMPLE, :].astype('float16')
                # Y: indicator of the belongings of the TF bin
                # 1st speaker or second speaker
                Y = np.array([sample_1 > sample_2, sample_1 < sample_2]).astype('bool')
                Y = np.transpose(Y, [1, 2, 0])
                VAD = speech_VAD[k:k + FRAMES_PER_SAMPLE, :].astype('bool')
                sample_dict = {'Sample': sample_mix,
                               'VAD': VAD,
                               'Target': Y}
                self.samples.append(sample_dict)
                k = k + FRAMES_PER_SAMPLE

        pickle.dump(self.samples, open(save_path, 'wb'))
        self.tot_samp = len(self.samples)
        np.random.shuffle(self.samples)

    def gen_batch(self):
        '''Output a batch of training samples'''
        n_begin = self.ind
        n_end = self.ind + self.batch_size
        if n_end >= self.tot_samp:
            self.ind = 0
            n_begin = self.ind
            n_end = self.ind + self.batch_size
            self.epoch += 1
            if self.epoch % 100 == 0:
                self.reinit()
        self.ind += self.batch_size
        return self.samples[n_begin:n_end]  

In [4]:
data_dir = 'data/train'
val_dir = 'data/test'

In [5]:
class DataGenerator2(object):
    def __init__(self, pkl_list, batch_size):
        '''pkl_list: .pkl files contaiing the data set'''
        self.ind = 0  # index of current reading position
        self.batch_size = batch_size
        self.samples = []
        self.epoch = 0
        # read in all the .pkl files
        for pkl in pkl_list:
            self.samples.extend(pickle.load(open(pkl, 'rb')))
        self.tot_samp = len(self.samples)
        print(self.tot_samp)
        print('samples')
        np.random.shuffle(self.samples)

    def gen_batch(self):
        # generate a batch of data
        n_begin = self.ind
        n_end = self.ind + self.batch_size
        if n_end >= self.tot_samp:
            # rewire the index
            self.ind = 0
            n_begin = self.ind
            n_end = self.ind + self.batch_size
            self.epoch += 1
            np.random.shuffle(self.samples)
        self.ind += self.batch_size
        return self.samples[n_begin:n_end]

In [6]:
class Model(object):
    def __init__(self, n_hidden, batch_size, p_keep_ff, p_keep_rc):
        '''n_hidden: number of hidden states
           p_keep_ff: forward keep probability
           p_keep_rc: recurrent keep probability'''
        self.n_hidden = n_hidden
        self.batch_size = batch_size
        self.p_keep_ff = p_keep_ff
        self.p_keep_rc = p_keep_rc
        # biases and weights for the last layer
        self.weights = {'out': tf.Variable(tf.random_normal([2 * n_hidden, EMBBEDDING_D * NEFF]))}
        self.biases = {'out': tf.Variable(tf.random_normal([EMBBEDDING_D * NEFF]))}

    def inference(self, x):
        '''The structure of the network'''
        # four layer of LSTM cell blocks
        with tf.variable_scope('BLSTM1') as scope:
            lstm_fw_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc) 
            lstm_fw_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_fw_cell, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            lstm_bw_cell = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_bw_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_bw_cell, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            outputs, _ = tf.nn.bidirectional_dynamic_rnn(
                lstm_fw_cell, lstm_bw_cell, x,
                sequence_length=[FRAMES_PER_SAMPLE] * self.batch_size,
                dtype=tf.float32)
            state_concate = tf.concat(outputs, 2)
        with tf.variable_scope('BLSTM2') as scope:
            lstm_fw_cell2 = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_fw_cell2 = tf.nn.rnn_cell.DropoutWrapper(
                lstm_fw_cell2, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            lstm_bw_cell2 = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_bw_cell2 = tf.nn.rnn_cell.DropoutWrapper(
                lstm_bw_cell2, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            outputs2, _ = tf.nn.bidirectional_dynamic_rnn(
                lstm_fw_cell2, lstm_bw_cell2, state_concate,
                sequence_length=[FRAMES_PER_SAMPLE] * self.batch_size,
                dtype=tf.float32)
            state_concate2 = tf.concat(outputs2, 2)
        with tf.variable_scope('BLSTM3') as scope:
            lstm_fw_cell3 = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_fw_cell3 = tf.nn.rnn_cell.DropoutWrapper(
                lstm_fw_cell3, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            lstm_bw_cell3 = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_bw_cell3 = tf.nn.rnn_cell.DropoutWrapper(
                lstm_bw_cell3, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            outputs3, _ = tf.nn.bidirectional_dynamic_rnn(
                lstm_fw_cell3, lstm_bw_cell3, state_concate2,
                sequence_length=[FRAMES_PER_SAMPLE] * self.batch_size,
                dtype=tf.float32)
            state_concate3 = tf.concat(outputs3, 2)
        with tf.variable_scope('BLSTM4') as scope:
            lstm_fw_cell4 = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_fw_cell4 = tf.nn.rnn_cell.DropoutWrapper(
                lstm_fw_cell4, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            lstm_bw_cell4 = tf.contrib.rnn.LayerNormBasicLSTMCell(
                self.n_hidden, layer_norm=False,
                dropout_keep_prob=self.p_keep_rc)
            lstm_bw_cell4 = tf.nn.rnn_cell.DropoutWrapper(
                lstm_bw_cell4, input_keep_prob=1,
                output_keep_prob=self.p_keep_ff)
            outputs4, _ = tf.nn.bidirectional_dynamic_rnn(
                lstm_fw_cell4, lstm_bw_cell4, state_concate3,
                sequence_length=[FRAMES_PER_SAMPLE] * self.batch_size,
                dtype=tf.float32)
            state_concate4 = tf.concat(outputs4, 2)
        # one layer of embedding output with tanh activation function
        out_concate = tf.reshape(state_concate4, [-1, self.n_hidden * 2])
        emb_out = tf.matmul(out_concate,self.weights['out']) + self.biases['out']
        emb_out = tf.nn.tanh(emb_out)
        reshaped_emb = tf.reshape(emb_out, [-1, NEFF, EMBBEDDING_D])
        # normalization before output
        normalized_emb = tf.nn.l2_normalize(reshaped_emb, 2)
        return normalized_emb

    def loss(self, embeddings, Y, VAD):
        '''Defining the loss function'''
        embeddings_rs = tf.reshape(embeddings, shape=[-1, EMBBEDDING_D])
        VAD_rs = tf.reshape(VAD, shape=[-1])
        # get the embeddings with active VAD
        embeddings_rsv = tf.transpose(tf.multiply(tf.transpose(embeddings_rs), VAD_rs))
        embeddings_v = tf.reshape(embeddings_rsv, [-1, FRAMES_PER_SAMPLE * NEFF, EMBBEDDING_D])
        # get the Y(speaker indicator function) with active VAD
        Y_rs = tf.reshape(Y, shape=[-1, 2])
        Y_rsv = tf.transpose(tf.multiply(tf.transpose(Y_rs), VAD_rs))
        Y_v = tf.reshape(Y_rsv, shape=[-1, FRAMES_PER_SAMPLE * NEFF, 2])
        # fast computation format of the embedding loss function
        loss_batch = tf.nn.l2_loss(tf.matmul(tf.transpose(embeddings_v, [0, 2, 1]), embeddings_v)) - \
            2 * tf.nn.l2_loss(tf.matmul(tf.transpose(embeddings_v, [0, 2, 1]), Y_v)) + \
            tf.nn.l2_loss(tf.matmul(tf.transpose(Y_v, [0, 2, 1]), Y_v))

        loss_v = (loss_batch) / self.batch_size
        tf.summary.scalar('loss', loss_v)
        return loss_v

    def train(self, loss, lr):
        '''Optimizer'''
        optimizer = tf.train.AdamOptimizer(learning_rate=lr,beta1=0.9,beta2=0.999,epsilon=1e-8)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 200)
        train_op = optimizer.apply_gradients(zip(gradients, v))
        return train_op

In [8]:
pkl_list = ['data/train/train_01.pkl']
val_list = ['data/test/val.pkl']
sum_dir = 'sum'
train_dir = 'train'

lr = 1e-6
n_hidden = 300
max_steps = 900100
batch_size = 16

In [7]:
def train():
    with tf.Graph().as_default():
        # dropout keep probability
        p_keep_ff = tf.placeholder(tf.float32, shape=None)
        p_keep_rc = tf.placeholder(tf.float32, shape=None)
        # generator for training set and validation set
        data_generator = DataGenerator2(pkl_list, batch_size)
        val_generator = DataGenerator2(val_list, batch_size)
        print('datagenerator finish')
        # placeholder for input log spectrum, VAD info.,
        # and speaker indicator function
        in_data = tf.placeholder(tf.float32, shape=[batch_size, FRAMES_PER_SAMPLE, NEFF])
        VAD_data = tf.placeholder(tf.float32, shape=[batch_size, FRAMES_PER_SAMPLE, NEFF])
        Y_data = tf.placeholder(tf.float32, shape=[batch_size, FRAMES_PER_SAMPLE, NEFF, 2])
        # init the model
        BiModel = Model(n_hidden, batch_size, p_keep_ff, p_keep_rc)
        # build the net structure
        embedding = BiModel.inference(in_data)
        print('model build finish')
        Y_data_reshaped = tf.reshape(Y_data, [-1, NEFF, 2])
        VAD_data_reshaped = tf.reshape(VAD_data, [-1, NEFF])
        # compute the loss
        loss = BiModel.loss(embedding, Y_data_reshaped, VAD_data_reshaped)
        print('loss build finish')
        # get the train operation
        train_op = BiModel.train(loss, lr)
        saver = tf.train.Saver(tf.global_variables())
        summary_op = tf.summary.merge_all()
        sess = tf.Session()

        # either train from scratch or a trained model
        saver.restore(sess, 'train/484000model.ckpt-484000')
        val_loss = np.fromfile('val_loss').tolist()
        init_step = 484001
        #init = tf.global_variables_initializer()
        #sess.run(init)
        #init_step = 0

        summary_writer = tf.summary.FileWriter(sum_dir, sess.graph)

        last_epoch = data_generator.epoch

        for step in range(init_step, max_steps):
            start_time = time.time()
            data_batch = data_generator.gen_batch()
            # concatenate the samples into batch data
            in_data_np = np.concatenate([np.reshape(item['Sample'], [1, FRAMES_PER_SAMPLE, NEFF]) for item in data_batch])
            VAD_data_np = np.concatenate([np.reshape(item['VAD'], [1, FRAMES_PER_SAMPLE, NEFF]) for item in data_batch])
            VAD_data_np = VAD_data_np.astype('int')
            Y_data_np = np.concatenate([np.reshape(item['Target'], [1, FRAMES_PER_SAMPLE, NEFF, 2]) for item in data_batch])
            Y_data_np = Y_data_np.astype('int')
            # train the model
            loss_value, _, summary_str = sess.run([loss, train_op, summary_op],feed_dict={in_data: in_data_np, \
                                                                                          VAD_data: VAD_data_np, \
                                                                                          Y_data: Y_data_np, \
                                                                                          p_keep_ff: 1 - P_DROPOUT_FF, \
                                                                                          p_keep_rc: 1 - P_DROPOUT_RC})
            summary_writer.add_summary(summary_str, step)
            duration = time.time() - start_time
            assert not np.isnan(loss_value)
            if step % 100 == 0:
                # show training progress every 100 steps
                num_examples_per_step = batch_size
                examples_per_sec = num_examples_per_step / duration
                sec_per_batch = float(duration)

                format_str = (
                    '%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                    'sec/batch, epoch %d)')
                print (format_str % (datetime.now(), step, loss_value,
                                     examples_per_sec, sec_per_batch,
                                     data_generator.epoch))
            if step % 4000 == 0 and step != 0:
                # save model every 4000 steps
                checkpoint_path = os.path.join(train_dir, str(step)+'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
                print('step: ', step, 'save model: ', train_dir+str(step)+'model.ckpt')

            if last_epoch != data_generator.epoch:
                # doing validation every training epoch
                print('Doing validation')
                val_epoch = val_generator.epoch
                count = 0
                loss_sum = 0
                val_loss = []
                # average the validation loss
                while(val_epoch == val_generator.epoch):
                    count += 1
                    data_batch = val_generator.gen_batch()
                    in_data_np = np.concatenate([np.reshape(item['Sample'],[1, FRAMES_PER_SAMPLE, NEFF]) for item in data_batch])
                    VAD_data_np = np.concatenate([np.reshape(item['VAD'],[1, FRAMES_PER_SAMPLE, NEFF]) for item in data_batch])
                    VAD_data_np = VAD_data_np.astype('int')
                    Y_data_np = np.concatenate([np.reshape(item['Target'],[1, FRAMES_PER_SAMPLE, NEFF, 2]) for item in data_batch])
                    Y_data_np = Y_data_np.astype('int')
                    loss_value, = sess.run(
                        [loss],
                        feed_dict={in_data: in_data_np,
                                   VAD_data: VAD_data_np,
                                   Y_data: Y_data_np,
                                   p_keep_ff: 1,
                                   p_keep_rc: 1})
                    loss_sum += loss_value
                val_loss.append(loss_sum / count)
                print ('validation loss: %.3f' % (loss_sum / count))
                np.array(val_loss).tofile('val_loss')

            last_epoch = data_generator.epoch

In [ ]:
print('%s start' % datetime.now())
train()

2020-02-18 10:28:23.892747 start
2437
samples
477
samples
datagenerator finish
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
model buil

Doing validation
validation loss: 5191909.207
2020-02-18 11:24:35.247405: step 444300, loss = 1241617.00 (23.4 examples/sec; 0.684 sec/batch, epoch 28)
2020-02-18 11:25:43.085011: step 444400, loss = 1527234.50 (23.9 examples/sec; 0.669 sec/batch, epoch 28)
Doing validation
validation loss: 5325738.138
2020-02-18 11:26:59.733623: step 444500, loss = 1478615.50 (23.2 examples/sec; 0.688 sec/batch, epoch 29)
Doing validation
validation loss: 5264495.121
2020-02-18 11:28:17.256649: step 444600, loss = 1686671.00 (23.9 examples/sec; 0.670 sec/batch, epoch 30)
2020-02-18 11:29:25.636848: step 444700, loss = 1131297.50 (23.9 examples/sec; 0.668 sec/batch, epoch 30)
Doing validation
validation loss: 5184003.155
2020-02-18 11:30:44.100481: step 444800, loss = 1400299.50 (23.5 examples/sec; 0.680 sec/batch, epoch 31)
Doing validation
validation loss: 5293459.776
2020-02-18 11:32:01.407537: step 444900, loss = 1111875.50 (23.8 examples/sec; 0.671 sec/batch, epoch 32)
2020-02-18 11:33:09.869207: 

2020-02-18 12:43:30.941131: step 450300, loss = 1275813.50 (17.3 examples/sec; 0.927 sec/batch, epoch 67)
Doing validation
validation loss: 5286602.810
2020-02-18 12:45:17.457333: step 450400, loss = 1332668.50 (17.4 examples/sec; 0.918 sec/batch, epoch 68)
Doing validation
validation loss: 5252378.483
2020-02-18 12:47:02.574227: step 450500, loss = 1569339.00 (17.8 examples/sec; 0.901 sec/batch, epoch 69)
2020-02-18 12:48:38.677253: step 450600, loss = 1427783.00 (17.6 examples/sec; 0.911 sec/batch, epoch 69)
Doing validation
validation loss: 5254521.500
2020-02-18 12:50:25.356001: step 450700, loss = 1068102.00 (17.2 examples/sec; 0.930 sec/batch, epoch 70)
Doing validation
validation loss: 5275628.879
2020-02-18 12:52:10.045823: step 450800, loss = 1477670.00 (17.5 examples/sec; 0.914 sec/batch, epoch 71)
2020-02-18 12:53:45.736962: step 450900, loss = 1531982.00 (17.4 examples/sec; 0.920 sec/batch, epoch 71)
Doing validation
validation loss: 5246236.914
2020-02-18 12:55:33.225537: 

Doing validation
validation loss: 5271215.534
2020-02-18 14:07:50.994026: step 456300, loss = 1093027.00 (23.2 examples/sec; 0.691 sec/batch, epoch 107)
2020-02-18 14:09:01.412447: step 456400, loss = 1762834.00 (23.0 examples/sec; 0.697 sec/batch, epoch 107)
Doing validation
validation loss: 5181123.966
2020-02-18 14:10:24.059294: step 456500, loss = 1272251.00 (22.1 examples/sec; 0.724 sec/batch, epoch 108)
Doing validation
validation loss: 5289301.810
2020-02-18 14:11:45.858272: step 456600, loss = 1226929.50 (21.8 examples/sec; 0.734 sec/batch, epoch 109)
2020-02-18 14:12:58.401861: step 456700, loss = 1302077.00 (22.3 examples/sec; 0.716 sec/batch, epoch 109)
Doing validation
validation loss: 5246909.690
2020-02-18 14:14:21.759148: step 456800, loss = 1362764.00 (22.6 examples/sec; 0.709 sec/batch, epoch 110)
Doing validation
validation loss: 5248762.138
2020-02-18 14:15:42.962297: step 456900, loss = 1023209.00 (23.6 examples/sec; 0.678 sec/batch, epoch 111)
2020-02-18 14:16:51.2

2020-02-18 15:37:39.038991: step 462200, loss = 1008576.00 (18.7 examples/sec; 0.857 sec/batch, epoch 146)
2020-02-18 15:39:04.574274: step 462300, loss = 1448372.00 (18.7 examples/sec; 0.855 sec/batch, epoch 146)
Doing validation
validation loss: 5239467.397
2020-02-18 15:40:42.188300: step 462400, loss = 1022143.00 (18.4 examples/sec; 0.868 sec/batch, epoch 147)
Doing validation
validation loss: 5260297.172
2020-02-18 15:42:19.613784: step 462500, loss = 1013958.00 (18.4 examples/sec; 0.868 sec/batch, epoch 148)
2020-02-18 15:43:44.909674: step 462600, loss = 1276712.00 (18.5 examples/sec; 0.864 sec/batch, epoch 148)
Doing validation
validation loss: 5213683.414
2020-02-18 15:45:23.623258: step 462700, loss = 1447262.00 (18.6 examples/sec; 0.859 sec/batch, epoch 149)
2020-02-18 15:46:48.921174: step 462800, loss = 1259020.00 (18.6 examples/sec; 0.862 sec/batch, epoch 149)
Doing validation
validation loss: 5286733.397
2020-02-18 15:48:26.330707: step 462900, loss = 1214597.00 (18.7 ex

Doing validation
validation loss: 5180659.655
2020-02-18 18:53:07.808348: step 468000, loss = 1298192.00 (23.0 examples/sec; 0.695 sec/batch, epoch 184)
step:  468000 save model:  train468000model.ckpt
2020-02-18 18:54:22.073377: step 468100, loss = 1126358.00 (22.9 examples/sec; 0.700 sec/batch, epoch 184)
Doing validation
validation loss: 5294618.724
2020-02-18 18:55:41.712396: step 468200, loss = 1358230.00 (22.9 examples/sec; 0.700 sec/batch, epoch 185)
Doing validation
validation loss: 5239980.966
2020-02-18 18:57:01.193719: step 468300, loss = 1291491.50 (23.2 examples/sec; 0.690 sec/batch, epoch 186)
2020-02-18 18:58:10.940063: step 468400, loss = 1160469.00 (23.2 examples/sec; 0.689 sec/batch, epoch 186)
Doing validation
validation loss: 5241100.914
2020-02-18 18:59:30.632101: step 468500, loss = 1211928.00 (22.7 examples/sec; 0.706 sec/batch, epoch 187)
Doing validation
validation loss: 5259003.862
2020-02-18 19:00:50.296123: step 468600, loss = 1383826.00 (22.8 examples/sec; 

Doing validation
validation loss: 5181111.241
2020-02-18 20:09:37.795861: step 473900, loss = 1532977.00 (15.5 examples/sec; 1.030 sec/batch, epoch 223)
2020-02-18 20:11:35.381224: step 474000, loss = 1296577.00 (8.0 examples/sec; 2.005 sec/batch, epoch 223)
Doing validation
validation loss: 5324766.017
2020-02-18 20:13:38.822794: step 474100, loss = 1020729.00 (14.1 examples/sec; 1.137 sec/batch, epoch 224)
2020-02-18 20:15:34.754171: step 474200, loss = 1319192.00 (13.7 examples/sec; 1.167 sec/batch, epoch 224)
Doing validation
validation loss: 5231594.707
2020-02-18 20:17:38.064468: step 474300, loss = 1063732.00 (16.7 examples/sec; 0.958 sec/batch, epoch 225)
Doing validation
validation loss: 5275605.621
2020-02-18 20:19:38.211524: step 474400, loss = 1160806.50 (15.8 examples/sec; 1.013 sec/batch, epoch 226)
2020-02-18 20:21:23.645751: step 474500, loss = 1105497.00 (16.4 examples/sec; 0.977 sec/batch, epoch 226)
Doing validation
validation loss: 5268291.172
2020-02-18 20:23:23.35

Doing validation
validation loss: 5219764.397
2020-02-18 21:42:41.796647: step 479900, loss = 1152811.50 (23.2 examples/sec; 0.689 sec/batch, epoch 262)
Doing validation
validation loss: 5272149.121
2020-02-18 21:43:59.105768: step 480000, loss = 1419975.00 (23.7 examples/sec; 0.674 sec/batch, epoch 263)
step:  480000 save model:  train480000model.ckpt
2020-02-18 21:45:10.206341: step 480100, loss = 1379899.00 (23.6 examples/sec; 0.679 sec/batch, epoch 263)
Doing validation
validation loss: 5240731.362
2020-02-18 21:46:27.476165: step 480200, loss = 1589090.00 (23.3 examples/sec; 0.687 sec/batch, epoch 264)
Doing validation
validation loss: 5243627.828
2020-02-18 21:47:44.860531: step 480300, loss = 1287934.50 (23.4 examples/sec; 0.684 sec/batch, epoch 265)
2020-02-18 21:48:53.045400: step 480400, loss = 1416690.50 (23.8 examples/sec; 0.672 sec/batch, epoch 265)
Doing validation
validation loss: 5244751.310
2020-02-18 21:50:10.508270: step 480500, loss = 1058832.00 (23.5 examples/sec; 